# QUAKE

## 1.  Оценка качества работы Quake

Исправление ридов с помощью Quake

In [23]:
! python2 /Johnny/students/NGS/data/6/Quake/bin/quake.py correct -h

Usage: quake.py [options]

Options:
  -h, --help        show this help message and exit
  -r READSF         Fastq file of reads
  -f READS_LISTF    File containing fastq file names, one per line or two per
                    line for paired end reads.
  -k K              Size of k-mers to correct
  -p PROC           Number of processes [default: 4]
  -q QUALITY_SCALE  Quality value ascii scale, generally 64 or 33. If not
                    specified, it will guess.
  --no_count        Kmers are already counted and in expected file [reads
                    file].qcts or [reads file].cts [default: False]
  --no_cut          Coverage model is optimized and cutoff was printed to
                    expected file cutoff.txt [default: False]
  --int             Kmers were counted as integers w/o the use of quality
                    values [default: False]
  --ratio=RATIO     Likelihood ratio to set trusted/untrusted cutoff.
                    Generally set between 10-1000 with lower n

In [4]:
! python2 /Johnny/students/NGS/data/6/Quake/bin/quake.py -f reads_files -k 15

Guessing quality values are on ascii 33 scale
Processing sequences...
..2763204 sequences processed, 276320400 bp scanned
7167988 total distinct mers
7167988 mers occur at least 0 times
initial  value 43342.925353 
iter  10 value 36964.156085
iter  20 value 34572.732523
iter  30 value 34353.327472
iter  40 value 33674.460444
iter  50 value 33653.583779
iter  60 value 33571.016252
iter  70 value 33520.531491
iter  80 value 33513.192688
iter  90 value 33508.255598
iter 100 value 33441.237694
iter 110 value 33407.173027
iter 120 value 33402.812959
iter 130 value 33400.796830
iter 140 value 33398.160523
iter 150 value 33396.469849
iter 160 value 33391.483544
iter 170 value 33389.770428
iter 180 value 33388.349378
iter 190 value 33387.104623
iter 200 value 33386.150163
iter 210 value 33383.247571
iter 220 value 33382.092360
iter 230 value 33379.993314
iter 240 value 33373.526364
final  value 33372.997786 
converged
value: 33373 
$zp.copy
[1] 3.133766

$p.e
[1] 0.9441405

$shape.e
[1] 0.2921

Выранивание ридов на геном. 

In [ ]:
! bwa index /Johnny/students/NGS/data/6/MG1655-K12.first400K.fasta -p ./MG

In [ ]:
! bwa mem MG /Johnny/students/NGS/data/6/ecoli_400K_err_1.fastq /Johnny/students/NGS/data/6/ecoli_400K_err_2.fastq > MG_err.sam

In [ ]:
! bwa mem MG ecoli_400K_err_1.cor.fastq ecoli_400K_err_2.cor.fastq > MG_cor.sam

Подсчет False positives, False negatives, True negatives для конкретного рида. 

Смотрим куда выравнились оба рида. Если в разные места или не выравнились, то не берем риды в расчет.

Проходим по всем элементам. Если два нуклеотида выравнились в одно и тоже места, то взависимотси от того, кто из них правильный увеличиваем нужное значение. 

Если оба нуклеотида не выравнились, то засчитывем как ошибка для обоих, если не выравнился один из нуклеотидов то только ошибка для того, кто не выравнился. 

Если нуклеотид из референса не выравнился хотя бы на одну из последовательностей, то выравнивание с данным нуклеотидом в расчет не берем. 

In [15]:
import pysam
import matplotlib.pyplot as plt
%matplotlib inline
from Bio import SeqIO
import pandas as pd

In [42]:
def calculateStatistic(record1, record2, ref):
    if record1.is_unmapped or record2.is_unmapped:
        return (0, 0, 0)
    if record1.reference_start != record2.reference_start:
        return (0, 0, 0)
    
    FP = 0
    FN = 0
    TN = 0
        
    cigar1 = record1.cigartuples
    cigar2 = record2.cigartuples
    rpos = record1.reference_start
    seq1 = record1.query_sequence
    seq2 = record2.query_sequence
    spos1 = 0
    spos2 = 0
    ci1 = 0
    ci2 = 0
    cnt1 = 0
    cnt2 = 0
    while (ci1 < len(cigar1)) or (ci2 < len(cigar2)):
        if (ci1 < len(cigar1) and (cigar1[ci1][0] == 0 or cigar1[ci1][0] == 7 or cigar1[ci1][0] == 8)) and (ci2 < len(cigar2) and (cigar2[ci2][0] == 0 or cigar2[ci2][0] == 7 or cigar2[ci2][0] == 8)):
            if (seq1[spos1] == ref[rpos]) and (seq2[spos2] != ref[rpos]):
                FN += 1
            elif (seq1[spos1] != ref[rpos]) and (seq2[spos2] == ref[rpos]):
                TN += 1
            elif (seq1[spos1] != ref[rpos]) and (seq2[spos2] != ref[rpos]):
                FP += 1
            cnt1 += 1
            cnt2 += 1
            rpos += 1
            spos1 += 1
            spos2 += 1
        elif (ci1 < len(cigar1) and (cigar1[ci1][0] == 1 or cigar1[ci1][0] == 4)) and  ((ci2 < len(cigar2)) and (cigar2[ci2][0] == 1 or cigar2[ci2][0] == 4)):
            FP += 1
            cnt1 += 1
            cnt2 += 1
            spos1 += 1
            spos2 += 1
        elif (ci1 < len(cigar1) and (cigar1[ci1][0] == 1 or cigar1[ci1][0] == 4)):
            TN += 1
            cnt1 += 1
            spos1 += 1
        elif (ci2 < len(cigar2) and (cigar2[ci2][0] == 1 or cigar2[ci2][0] == 4)):
            FN += 1
            cnt2 += 1
            spos2 += 1
        else:
            if (ci1 < len(cigar1) and (cigar1[ci1][0] == 0 or cigar1[ci1][0] == 7 or cigar1[ci1][0] == 8)):
                spos1 += 1
            if (ci2 < len(cigar2) and (cigar2[ci2][0] == 0 or cigar2[ci2][0] == 7 or cigar2[ci2][0] == 8)):
                spos2 += 1
            if (ci1 < len(cigar1)):
                cnt1 += 1
            if (ci2 < len(cigar2)):
                cnt2 += 1
            rpos += 1
            
        if (ci1 < len(cigar1) and cnt1 == cigar1[ci1][1]):
            cnt1 = 0
            ci1 += 1
        if (ci2 < len(cigar2) and cnt2 == cigar2[ci2][1]):
            cnt2 = 0
            ci2 += 1
    return (FP, FN, TN)
    

In [45]:
def main(refFileName, oldFileName, newFileName):
    ref = ""
    for rec in SeqIO.parse(refFileName, "fasta"):
        ref = rec.seq
    oldSamFile = pysam.AlignmentFile(oldFileName, "r")
    newSamFile = pysam.AlignmentFile(newFileName, "r")
    
    FP = 0
    FN = 0
    TN = 0
    iter1 = oldSamFile.fetch()
    cnt = 0
    for record2 in newSamFile.fetch():
        if (cnt % 10000 == 0):
            print(cnt, record2.query_name)
        record1 = next(iter1)
        while (record1.query_name != record2.query_name):
            record1 = next(iter1)
        cFP, cFN, cTP = calculateStatistic(record1, record2, ref)
        FP += cFP
        FN += cFN
        TN += cTP
        cnt += 1
    
    print("False positives =", FP, " False negatives =", FN, " True negatives =", TN )
    return (FP, FN, TN)

In [ ]:
FP, FN, TN = main("/Johnny/students/NGS/data/6/MG1655-K12.first400K.fasta", "MG_err.sam", "MG_cor.sam")

In [47]:
print("False positives =", FP, " False negatives =", FN, " True negatives =", TN )

False positives = 70817  False negatives = 109  True negatives = 607796
